In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

### Import Data

In [ ]:
rating = pd.read_csv('BX-Book-Ratings.csv', sep=';', error_bad_lines=False, encoding="latin-1")

/Users/roethelchristine/final/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
user = pd.read_csv('BX-Users.csv', sep=';', error_bad_lines=False, encoding="latin-1")

In [ ]:
book = pd.read_csv('BX-Books.csv', sep=';', error_bad_lines=False, encoding="latin-1")

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/roethelchristine/final/lib/python3.9/site-packages/IPython/core/int

In [ ]:
book_rating = pd.merge(rating, book, on='ISBN')

In [ ]:
cols = ['Year-Of-Publication', 'Publisher', 'Book-Author', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']

In [ ]:
book_rating.drop(cols, axis=1, inplace=True)

In [ ]:
rating_count = (book_rating.
     groupby(by = ['Book-Title'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_book'})
     [['Book-Title', 'RatingCount_book']]
    )

In [ ]:
threshold = 25
rating_count = rating_count.query('RatingCount_book >= @threshold')

In [ ]:
user_rating = pd.merge(rating_count, book_rating, left_on='Book-Title', right_on='Book-Title', how='left')

In [ ]:
user_count = (user_rating.
     groupby(by = ['User-ID'])['Book-Rating'].
     count().
     reset_index().
     rename(columns = {'Book-Rating': 'RatingCount_user'})
     [['User-ID', 'RatingCount_user']]
    )

In [ ]:
threshold = 20
user_count = user_count.query('RatingCount_user >= @threshold')

In [ ]:
combined = user_rating.merge(user_count, left_on = 'User-ID', right_on = 'User-ID', how = 'inner')

In [ ]:
print('Number of unique books: ', combined['Book-Title'].nunique())
print('Number of unique users: ', combined['User-ID'].nunique())

Number of unique books:  5850
Number of unique users:  3192


## TensorFlow: Normalize the rating feature

In [ ]:
scaler = MinMaxScaler()
combined['Book-Rating'] = combined['Book-Rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(combined['Book-Rating'].values.reshape(-1,1)))
combined['Book-Rating'] = rating_scaled

### Then, build user, book matrix with three features:

In [ ]:
combined = combined.drop_duplicates(['User-ID', 'Book-Title'])
user_book_matrix = combined.pivot(index='User-ID', columns='Book-Title', values='Book-Rating')
user_book_matrix.fillna(0, inplace=True)
users = user_book_matrix.index.tolist()
books = user_book_matrix.columns.tolist()
user_book_matrix = user_book_matrix.to_numpy()

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
num_input = combined['Book-Title'].nunique()
num_hidden_1 = 10
num_hidden_2 = 5

X = tf.placeholder(tf.float64, [None, num_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1], dtype=tf.float64)),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2], dtype=tf.float64)),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1], dtype=tf.float64)),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input], dtype=tf.float64)),
}

biases = {
    'encoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'encoder_b2': tf.Variable(tf.random_normal([num_hidden_2], dtype=tf.float64)),
    'decoder_b1': tf.Variable(tf.random_normal([num_hidden_1], dtype=tf.float64)),
    'decoder_b2': tf.Variable(tf.random_normal([num_input], dtype=tf.float64)),
}

In [ ]:
def encoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']), biases['encoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']), biases['encoder_b2']))
    return layer_2

def decoder(x):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']), biases['decoder_b1']))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']), biases['decoder_b2']))
    return layer_2

In [ ]:
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)
y_pred = decoder_op
y_true = X

In [ ]:
loss = tf.losses.mean_squared_error(y_true, y_pred)
optimizer = tf.train.RMSPropOptimizer(0.03).minimize(loss)
eval_x = tf.placeholder(tf.int32, )
eval_y = tf.placeholder(tf.int32, )
pre, pre_op = tf.metrics.precision(labels=eval_x, predictions=eval_y)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()
pred_data = pd.DataFrame()

In [ ]:
with tf.Session() as session:
    epochs = 100
    batch_size = 35

    session.run(init)
    session.run(local_init)

    num_batches = int(user_book_matrix.shape[0] / batch_size)
    user_book_matrix = np.array_split(user_book_matrix, num_batches)
    
    for i in range(epochs):

        avg_cost = 0
        for batch in user_book_matrix:
            _, l = session.run([optimizer, loss], feed_dict={X: batch})
            avg_cost += l

        avg_cost /= num_batches

        print("epoch: {} Loss: {}".format(i + 1, avg_cost))

    user_book_matrix = np.concatenate(user_book_matrix, axis=0)

    preds = session.run(decoder_op, feed_dict={X: user_book_matrix})

    pred_data = pred_data.append(pd.DataFrame(preds))

    pred_data = pred_data.stack().reset_index(name='Book-Rating')
    pred_data.columns = ['User-ID', 'Book-Title', 'Book-Rating']
    pred_data['User-ID'] = pred_data['User-ID'].map(lambda value: users[value])
    pred_data['Book-Title'] = pred_data['Book-Title'].map(lambda value: books[value])
    
    keys = ['User-ID', 'Book-Title']
    index_1 = pred_data.set_index(keys).index
    index_2 = combined.set_index(keys).index

    top_ten_ranked = pred_data[~index_1.isin(index_2)]
    top_ten_ranked = top_ten_ranked.sort_values(['User-ID', 'Book-Rating'], ascending=[True, False])
    top_ten_ranked = top_ten_ranked.groupby('User-ID').head(10)

2022-01-07 08:52:37.692635: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


epoch: 1 Loss: 0.3479609063693455
epoch: 2 Loss: 0.2937775157637649
epoch: 3 Loss: 0.07109420311094804
epoch: 4 Loss: 0.0035767420015942594
epoch: 5 Loss: 0.003198640751089532
epoch: 6 Loss: 0.003111606632656121
epoch: 7 Loss: 0.002930433593004696
epoch: 8 Loss: 0.0028381019185941955
epoch: 9 Loss: 0.0027304790673552304
epoch: 10 Loss: 0.002721444797026669
epoch: 11 Loss: 0.002714794647245371
epoch: 12 Loss: 0.002709670021751343
epoch: 13 Loss: 0.0027056039467065053
epoch: 14 Loss: 0.0027023035211611417
epoch: 15 Loss: 0.002699574534934792
epoch: 16 Loss: 0.002697283211471206
epoch: 17 Loss: 0.0026953342437211958
epoch: 18 Loss: 0.0026936581624405725
epoch: 19 Loss: 0.0026922028212408933
epoch: 20 Loss: 0.0026909286798499936
epoch: 21 Loss: 0.0026898049547306293
epoch: 22 Loss: 0.002688807303566936
epoch: 23 Loss: 0.002687916430839143
epoch: 24 Loss: 0.002687116514297796
epoch: 25 Loss: 0.0026863949120065175
epoch: 26 Loss: 0.0026857409201521466
epoch: 27 Loss: 0.002685145950865942
epo

In [ ]:
top_ten_ranked.loc[top_ten_ranked['User-ID'] == 278582]

,User-ID,Book-Title,Book-Rating
18660405,278582,The Lovely Bones: A Novel,0.075071
18659952,278582,The Da Vinci Code,0.062520
18660710,278582,The Secret Life of Bees,0.046998
18657487,278582,Harry Potter and the Chamber of Secrets (Book 2),0.046004
18657491,278582,Harry Potter and the Prisoner of Azkaban (Book 3),0.043969
18658056,278582,Life of Pi,0.042014
18656352,278582,Bridget Jones's Diary,0.041977
18657493,278582,Harry Potter and the Sorcerer's Stone (Harry P...,0.039356
18660634,278582,The Red Tent (Bestselling Backlist),0.038365
18661051,278582,To Kill a Mockingbird,0.038035


In [ ]:
book_rating.loc[book_rating['User-ID'] == 278582].sort_values(by=['Book-Rating'], ascending=False)

,User-ID,ISBN,Book-Rating,Book-Title
174885,278582,0226848620,10,Chinese Bell Murders (Judge Dee Mysteries)
176582,278582,157566254X,10,"Skin Deep, Blood Red"
40008,278582,0441478123,10,The Left Hand of Darkness (Remembering Tomorrow)
174861,278582,0061044725,10,Search the Shadows
58156,278582,0451202503,10,The Songcatcher: A Ballad Novel
64570,278582,1400034779,10,The No. 1 Ladies' Detective Agency (Today Show...
175958,278582,0345350499,10,The Mists of Avalon
176314,278582,0449223558,9,Murdering Mr. Monti: A Merry Little Tale of Se...
174877,278582,0140277471,9,Blanche Cleans Up
176438,278582,0515136557,8,The Cat Who Brought Down the House
